Avance 2 - Proyecto Final

En este notebook, se presenta el desarrollo de un sistema de análisis de ventas. El objetivo es procesar datos de ventas, cargarlos en una base de datos MySQL y realizar análisis complejos para extraer insights sobre el rendimiento de productos, sucursales y el comportamiento de los clientes. Se utilizarán principios de Programación Orientada a Objetos, patrones de diseño y SQL avanzado para construir una solución robusta y escalable.

En la siguiente celda, importamos todas las librerías necesarias


In [ ]:
from src.database.db_connector import DatabaseConnector

db = DatabaseConnector()
session = db.get_session()
print("✅ Conexión exitosa")


✅ Conexión exitosa


In [ ]:
df = db.run_query("SELECT * FROM sales LIMIT 5")
df.head()

,sale_id,employee_id,customer_id,product_id,quantity,discount,total_price,sale_time,transaction_number
0,14,7,33133,406,9,0.00,63.00,39:52.0,VOP9A7Y4C5XSM2LLT0UJ\r
1,170,10,19039,377,5,0.00,50.00,33:40.0,VIIKLFPVD2D5U5E3NKKA\r
2,416,16,71212,351,19,0.00,304.00,40:57.4,QFA3F3LANPC6UVQBPYL2\r
3,486,23,93253,417,24,0.00,552.00,59:29.4,6H8SDUI9MCA1IXZ4UCUE\r
4,659,13,65151,147,17,0.20,221.00,27:58.9,1O062RUFV0LVAQ94G38J\r


Patrones de diseño
Definición de Clases y Patrones de Diseño
Patron factory

In [ ]:
from src.factories.model_factory import ModelFactory

row = {
    "CustomerID": 1,
    "FirstName": "Juan",
    "MiddleInitial": "M",
    "LastName": "Perez",
    "CityID": 1,
    "Address": "martinez 123",
}

customer = ModelFactory.create_customer(row)
print(customer.__dict__)


{'_customer_id': 1, '_first_name': 'Juan', '_middle_initial': 'M', '_last_name': 'Perez', '_city_id': 1, '_address': 'martinez 123'}


Implementación del patron Builder se mostrara con el SQL

5. Análisis de Ventas con SQL Avanzado

En esta sección, se realizan consultas complejas para responder a preguntas clave del negocio.

5.1. Ranking de Productos Más Vendidos por Categoría

Pregunta de Negocio: ¿Cuáles son los 3 productos más vendidos (en cantidad) para cada categoría?
*Técnica SQL:* Función de Ventana RANK() OVER (PARTITION BY ...).

In [ ]:
query_ranking = """
    SELECT
        CategoryName,
        ProductName,
        TotalQuantity,
        ProductRank
    FROM (
        SELECT
            c.CategoryName,
            p.ProductName,
            SUM(s.Quantity) AS TotalQuantity,
            RANK() OVER (PARTITION BY c.CategoryName ORDER BY SUM(s.Quantity) DESC) AS ProductRank
        FROM sales s
        JOIN products p ON s.ProductID = p.ProductID
        JOIN categories c ON p.CategoryID = c.CategoryID
        GROUP BY c.CategoryName, p.ProductName
    ) AS RankedProducts
    WHERE ProductRank <= 3;
"""
df_ranking = db.run_query(query_ranking)

print("Ranking de Productos:")
display(df_ranking)

5.2. Uso del `SalesQueryBuilder` para Consultas Dinámicas

Ahora, usamos nuestro `SalesQueryBuilder` para encontrar las ventas del cliente con ID 45 y una cantidad mínima de 2 unidades

In [ ]:
from src.builder.sale_query_builder import SalesQueryBuilder

builder = SalesQueryBuilder()
query_string, params = builder.with_customer(45).with_min_quantity(2).build()

print("Consulta Construida:", query_string)
print("Parámetros:", params)

df_builder_result = db.run_query(query_string, params)

display(df_builder_result)

5.2. Análisis de Gasto por Cliente (usando CTE)

*Pregunta de Negocio:* ¿Quiénes son nuestros clientes más valiosos en una ciudad específica? Queremos identificar a los clientes con el mayor gasto total para posibles campañas de fidelización.

*Técnica SQL:* Se utilizará una Expresión de Tabla Común (CTE) llamada CustomerSpending para primero calcular el gasto total y el número de compras por cliente. Luego, se unirá este resultado con la tabla de clientes para obtener sus nombres y filtrar por ciudad. El uso de un CTE hace que la consulta sea más legible y modular.

In [ ]:
query_customer_spending = """
    WITH CustomerSpending AS (
        SELECT
            CustomerID,
            SUM(TotalPrice) AS TotalSpent,
            COUNT(SalesID) AS NumberOfPurchases
        FROM sales
        GROUP BY CustomerID
    )
    SELECT
        c.CustomerID,
        c.FirstName,
        c.LastName,
        cs.TotalSpent,
        cs.NumberOfPurchases
    FROM customers c
    JOIN CustomerSpending cs ON c.CustomerID = cs.CustomerID
    WHERE c.CityID = 1 -- Filtramos por una ciudad de ejemplo
    ORDER BY cs.TotalSpent DESC
    LIMIT 10; -- Mostramos el top 10
"""

print("Ejecutando consulta para analizar el gasto por cliente:")
df_customer_spending = db.run_query(query_customer_spending)

display(df_customer_spending)

6. Uso de Vistas y Procedimientos Almacenados

Finalmente, demostramos el uso de los objetos SQL creados para simplificar y automatizar el acceso a los datos.

6.1. Consulta a la Vista `V_SalesOverview`

In [ ]:
df_view = db.run_query("SELECT * FROM V_SalesOverview LIMIT 10;")
display(df_view)

6.2. Llamada al Procedimiento Almacenado `SP_GetProductSales`

In [ ]:
df_sp = db.run_query("CALL SP_GetProductSales(101);")
display(df_sp)

TESTS

In [ ]:
!pytest -q test/ --tb=short

....                                                                     [100%]
4 passed in 0.04s
